In [85]:
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import json

Below we standardise file formatting

In [86]:
code_to_desc = {}
for year in range(2015, 2026):
    if year > 2022:
        df = pd.read_excel(f"./Data/Income/Gross_Pay_{year}.xlsx",sheet_name=1).iloc[3:]
    else:
        df = pd.read_excel(f"./Data/Income/Gross_Pay_{year}.xls",sheet_name=1).iloc[3:]
    df.columns = df.iloc[0]
    code_to_desc.update(df.set_index('Code')['Description'].to_dict())
    df = df.rename_axis("Council Code", axis=1)
    df = df.dropna(subset=['Code'])
    df = df.drop(columns=["Description", "(thousand)", "change", "Mean"], axis=1)
    df = df.rename(columns={'Median': "50"})
    df.set_index("Code", inplace=True)
    df = df.iloc[:, :-3]
    df.index.name = None
    df.to_csv(f"./Temp/Gross_Pay_{year}.csv", index=True)   

with open("./CleanedData/Council_to_Code.json", "w") as f:
    json.dump(code_to_desc, f, indent=4)

Next we load each dataframe into a list 

In [87]:
dfs = [pd.read_csv(f"./Temp/Gross_Pay_{year}.csv") for year in range(2016, 2026)]

for i, df in enumerate(dfs):
    df = dfs[i]
    df.set_index("Unnamed: 0", inplace=True)
    df = df.tail(-1)
    df.index.name = None
    df = df.rename_axis("Council Code", axis=1)
    dfs[i] = df

Next we take the common council codes for each dataframe so each dataframe has the same dimensions

In [88]:

common_idx = reduce(lambda a, b: a.intersection(b.index), dfs, dfs[0].index)
dfs = [df.loc[common_idx] for df in dfs]
dfs[-1]

Council Code,50,10,20,25,30,40,60,70,75,80,90
K02000001,32890,11425,18560,22060,24532,28591,38000,44500,48283,52809,69381
K03000001,32972,11456,18613,22112,24580,28646,38061,44629,48408,52929,69750
K04000001,32991,11424,18589,22094,24563,28627,38058,44677,48479,53162,70250
E92000001,33142,11439,18653,22187,24669,28769,38292,44962,48820,53630,71090
E12000001,29266,10727,17213,20252,22769,26092,33006,38097,41339,45254,56226
...,...,...,...,...,...,...,...,...,...,...,...
S12000029,31984,10510,16577,19698,22873,27610,36257,42531,45225,49383,x
S12000030,34244,13366,20888,23221,25654,30216,37413,44045,47657,50139,x
S12000039,30033,11013,16971,20496,22896,26576,34887,41103,43996,46589,x
S12000040,32535,12198,19715,23137,24994,28624,37660,43582,46888,50103,x


In [89]:
years = list(range(2016, 2026))
combined_df = pd.concat(dfs, axis=1)
combined_df.columns = pd.MultiIndex.from_product([years, dfs[0].columns], names=['Year', 'Analysis'])
combined_df


Year        2016                                                         \
Analysis      50    10     20     25     30     40     60     70     75   
K02000001  23084  7498  11952  14099  15914  19466  27209  32220  35211   
K03000001  23162  7500  11986  14153  15961  19524  27315  32334  35300   
K04000001  23178  7468  11958  14135  15944  19530  27385  32429  35461   
E92000001  23334  7465  11977  14184  16000  19640  27562  32602  35700   
E12000001  21177  7451  11647  13515  15188  18225  24747  28575  31314   
...          ...   ...    ...    ...    ...    ...    ...    ...    ...   
S12000029  23719  7863  11892  14312  16725  20378  27545  31739  32710   
S12000030  23057  7110  11385  13767  15065  18214  26520  31545  34261   
S12000039  22389  9818  13159  14110  15903  18810  25477  31032  33031   
S12000040  21350  8582  12679  14271  15734  18789  24596  28385  30948   
N92000002  20966  7324  11079  12871  14535  17794  24520  29046  31992   

Year              ...   2025                                                   \
Analysis      80  ...     10     20     25     30     40     60     70     75   
K02000001  38788  ...  11425  18560  22060  24532  28591  38000  44500  48283   
K03000001  38904  ...  11456  18613  22112  24580  28646  38061  44629  48408   
K04000001  39143  ...  11424  18589  22094  24563  28627  38058  44677  48479   
E92000001  39450  ...  11439  18653  22187  24669  28769  38292  44962  48820   
E12000001  34761  ...  10727  17213  20252  22769  26092  33006  38097  41339   
...          ...  ...    ...    ...    ...    ...    ...    ...    ...    ...   
S12000029  35267  ...  10510  16577  19698  22873  27610  36257  42531  45225   
S12000030      x  ...  13366  20888  23221  25654  30216  37413  44045  47657   
S12000039  35146  ...  11013  16971  20496  22896  26576  34887  41103  43996   
S12000040  33239  ...  12198  19715  23137  24994  28624  37660  43582  46888   
N92000002  35669  ...  10233  17441  20924  23429  27376  35623  41031  44531   

Year                     
Analysis      80     90  
K02000001  52809  69381  
K03000001  52929  69750  
K04000001  53162  70250  
E92000001  53630  71090  
E12000001  45254  56226  
...          ...    ...  
S12000029  49383      x  
S12000030  50139      x  
S12000039  46589      x  
S12000040  50103      x  
N92000002  48703  61143  

[378 rows x 110 columns]

In [90]:
combined_df = combined_df.rename(index=code_to_desc)
combined_df.index = combined_df.index.str.strip()
combined_df.sort_index(inplace=True)

In [91]:
combined_percentiles = combined_df.loc[:, [col for col in combined_df.columns if col[1] not in ['Mean']]]
combined_percentiles = combined_percentiles.apply(pd.to_numeric, errors='coerce')



In [92]:
combined_median = combined_df.loc[:, [col for col in combined_df.columns if col[1]  in ['50']]]
combined_median = combined_median.apply(pd.to_numeric, errors='coerce')

combined_median

Year,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
Analysis,50,50,50,50,50,50,50,50,50,50
Aberdeen City,26507.0,25349.0,26087.0,27435.0,27970.0,26844.0,29429.0,33319.0,35160.0,35483.0
Aberdeenshire,20456.0,20726.0,22702.0,23064.0,23814.0,23473.0,25061.0,26724.0,29736.0,31551.0
Adur,NaN,22762.0,22914.0,21447.0,22877.0,22977.0,23563.0,28029.0,27753.0,28238.0
Amber Valley,23197.0,24060.0,22683.0,24950.0,24450.0,23301.0,23924.0,26342.0,29414.0,31100.0
Angus,20467.0,20022.0,21112.0,NaN,21785.0,22779.0,25822.0,26604.0,28748.0,27520.0
...,...,...,...,...,...,...,...,...,...,...
Wychavon,20864.0,21401.0,22810.0,22525.0,22543.0,24343.0,NaN,29891.0,31721.0,32022.0
Wyre,18155.0,NaN,17902.0,19227.0,20064.0,20985.0,23043.0,24953.0,26691.0,27657.0
Wyre Forest,NaN,18181.0,18158.0,20783.0,20293.0,22665.0,23501.0,NaN,25605.0,27346.0


In [ ]:
#IMPUTATION  HERE (CANT HAVE NAN VALUES BEFORE TERCILES)

In [102]:
import numpy as np
import pandas as pd

# ensure MultiIndex columns
if not isinstance(combined_median.columns, pd.MultiIndex):
    raise ValueError("df.columns must be a MultiIndex")

new_cols = {}

for year in combined_median.columns.levels[0]:

    # all percentile columns for that year
    year_df = combined_median[year]

    # flatten values, drop NaN
    values = year_df.values.flatten()
    values = values[~np.isnan(values)]

    # compute tercile cutoffs
    low_cut = np.percentile(values, 33)
    high_cut = np.percentile(values, 67)

    # you can choose: mean across percentiles OR a single chosen percentile
    base_vals = year_df.mean(axis=1)

    tercile_series = base_vals.apply(
        lambda v: ( "Low" if v < low_cut else "Mid" if v < high_cut else "High" ))
    new_cols[(year, "Tercile")] = tercile_series

tercile_df = pd.DataFrame(new_cols, index=combined_median.index)
tercile_df.columns = pd.MultiIndex.from_tuples(tercile_df.columns, names=combined_median.columns.names)
df_with_tercile = pd.concat([combined_median, tercile_df], axis=1)
df_with_tercile = df_with_tercile.sort_index(axis=1)
df_with_tercile = df_with_tercile.drop(columns=df_with_tercile.columns[df_with_tercile.columns.get_level_values(1) == "50"])
df_with_tercile.to_csv(f"./CleanedData/Income_Tercile_Map.csv", index=True)   



In [94]:
df_long = combined_percentiles.stack(level=[0,1]).reset_index()
df_long.columns = ['Council', 'Year', 'Percentile', 'Revenue']

# Convert to numeric
df_long['Year'] = pd.to_numeric(df_long['Year'])
df_long['Percentile'] = pd.to_numeric(df_long['Percentile'])
df_long['Revenue'] = pd.to_numeric(df_long['Revenue'])

fig = go.Figure()

x_min, x_max = df_long['Year'].min(), df_long['Year'].max()
y_min, y_max = df_long['Percentile'].min(), df_long['Percentile'].max()
z_min, z_max = df_long['Revenue'].min(), df_long['Revenue'].max()


councils = df_long['Council'].unique()
for i, council in enumerate(councils):
    df_c = df_long[df_long['Council'] == council]
    fig.add_trace(go.Scatter3d(x=df_c['Year'],y=df_c['Percentile'],z=df_c['Revenue'],mode='markers',marker=dict(size=5),name=council, visible=(i==0)))

buttons = []
for i, council in enumerate(councils):
    visible = [False]*len(councils)
    visible[i] = True
    buttons.append(dict(label=council, method="update", args=[{"visible": visible}, {"title": f"Region: {council}"}]))

fig.update_layout(
    updatemenus=[dict(active=0, buttons=buttons, x=1.1, y=0.8)],
    scene=dict(
        xaxis=dict(title='Year', range=[x_min, x_max]),
        yaxis=dict(title='Percentile', range=[y_min, y_max]),
        zaxis=dict(title='Revenue', range=[z_min, z_max]),
    ),
    title="Income Percentiles By Council By Region By Year"
)

fig.show()


/tmp/ipykernel_38011/4040778914.py:1: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.

